In [1]:
import gibbs
import numpy as np

In [2]:
N = 3  # neurons
K = 2  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10


pi=np.array([1,0])

gamma=np.array([[0.9,0.1],
                [0.1,0.9]])

r=np.array([[1,0],
            [0,1],
            [1,0],
            [0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=2)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

Data is generated according to an HMM with the following parameters:
Pi:
 [1 0]
Gamma:
 [[0.9 0.1]
 [0.1 0.9]]
R:
 [[0.5 0. ]
 [0.  0.5]
 [0.5 0. ]
 [0.  0.5]]
50000 data points are generated, which are:
Generated firings: [2 2 2 ... 0 0 2]
Generated states: [0 0 0 ... 0 0 0]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.99541267 0.00458733]
Estimated Gamma =  [[0.90883388 0.09116612]
 [0.08909989 0.91090011]]
Estimated R =  [[0.48548494 0.00741732]
 [0.01201551 0.4983155 ]
 [0.49186804 0.00703504]
 [0.01063151 0.48723213]]
Rep 1...
##########################
Estimated values for the parameters
Estimated Pi =  [0.2531916 0.7468084]
Estimated Gamma =  [[0.59540691 0.40459309]
 [0.67733746 0.32266254]]
Estimated R =  [[0.28438228 0.17705781]
 [0.32473635 0.14503013]
 [0.14249972 0.42306854]
 [0.24838165 0.25484352]]
Rep 2...
##########################
Estimated values for the parameters
Estimated Pi =  [0.99790942 0.00209058]
Estimated Gamma =

In [3]:
N = 3  # neurons
K = 4  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0])
gamma=np.array([[0.85,0.05,0.05,0.05],
                [0.05,0.85,0.05,0.05],
                [0.05,0.05,0.85,0.05],
                [0.05,0.05,0.05,0.85]])
r=np.array([[1,0,1,0],
            [0,1,1,0],
            [1,0,0,1],
            [0,1,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=4)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)


Data is generated according to an HMM with the following parameters:
Pi:
 [1 0 0 0]
Gamma:
 [[0.85 0.05 0.05 0.05]
 [0.05 0.85 0.05 0.05]
 [0.05 0.05 0.85 0.05]
 [0.05 0.05 0.05 0.85]]
R:
 [[0.5 0.  0.5 0. ]
 [0.  0.5 0.5 0. ]
 [0.5 0.  0.  0.5]
 [0.  0.5 0.  0.5]]
50000 data points are generated, which are:
Generated firings: [0 2 1 ... 3 1 3]
Generated states: [0 0 2 ... 1 1 1]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.18246565 0.56692708 0.11644085 0.13416642]
Estimated Gamma =  [[1.02959698e-01 5.08410517e-04 3.87639743e-01 5.08892148e-01]
 [5.61087190e-02 7.45821304e-01 1.01933185e-01 9.61367927e-02]
 [1.60218081e-01 2.20738253e-01 5.77075905e-01 4.19677604e-02]
 [4.03960710e-01 3.08417889e-01 1.19075955e-01 1.68545446e-01]]
Estimated R =  [[0.04722227 0.48181839 0.08973341 0.06663005]
 [0.07958056 0.40685933 0.15223264 0.13229912]
 [0.43984615 0.06943854 0.17437477 0.73127959]
 [0.43335102 0.04188374 0.58365918 0.06979124]]
Rep 1..

In [4]:
N = 3  # neurons
K = 5  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0])
gamma=np.array([[0.8,0.05,0.05,0.05,0.05],
                [0.05,0.8,0.05,0.05,0.05],
                [0.05,0.05,0.8,0.05,0.05],
                [0.05,0.05,0.05,0.8,0.05],
                [0.05,0.05,0.05,0.05,0.8]])
r=np.array([[1,0,1,0,1],
            [0,1,1,0,0],
            [1,0,0,1,1],
            [0,1,0,1,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=5)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)


Data is generated according to an HMM with the following parameters:
Pi:
 [1 0 0 0 0]
Gamma:
 [[0.8  0.05 0.05 0.05 0.05]
 [0.05 0.8  0.05 0.05 0.05]
 [0.05 0.05 0.8  0.05 0.05]
 [0.05 0.05 0.05 0.8  0.05]
 [0.05 0.05 0.05 0.05 0.8 ]]
R:
 [[0.5        0.         0.5        0.         0.33333333]
 [0.         0.5        0.5        0.         0.        ]
 [0.5        0.         0.         0.5        0.33333333]
 [0.         0.5        0.         0.5        0.33333333]]
50000 data points are generated, which are:
Generated firings: [2 0 0 ... 0 2 0]
Generated states: [0 0 2 ... 0 0 0]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.11221228 0.26734028 0.3366089  0.20449919 0.07933935]
Estimated Gamma =  [[0.09586571 0.1922322  0.19579756 0.18084674 0.33525779]
 [0.05570008 0.01710529 0.47206039 0.26569931 0.18943492]
 [0.46844947 0.00770398 0.06288182 0.22652934 0.23443539]
 [0.27701487 0.09370166 0.24140624 0.37039874 0.01747849]
 [0.34064467 0.

In [5]:
N = 3  # neurons
K = 6  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0])
gamma=np.array([[0.75,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.75,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.75,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.75,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.75,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.75]])
r=np.array([[1,0,1,0,1,1],
            [0,1,1,0,0,1],
            [1,0,0,1,1,0],
            [0,1,0,1,1,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=6)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

Data is generated according to an HMM with the following parameters:
Pi:
 [1 0 0 0 0 0]
Gamma:
 [[0.75 0.05 0.05 0.05 0.05 0.05]
 [0.05 0.75 0.05 0.05 0.05 0.05]
 [0.05 0.05 0.75 0.05 0.05 0.05]
 [0.05 0.05 0.05 0.75 0.05 0.05]
 [0.05 0.05 0.05 0.05 0.75 0.05]
 [0.05 0.05 0.05 0.05 0.05 0.75]]
R:
 [[0.5        0.         0.5        0.         0.33333333 0.33333333]
 [0.         0.5        0.5        0.         0.         0.33333333]
 [0.5        0.         0.         0.5        0.33333333 0.        ]
 [0.         0.5        0.         0.5        0.33333333 0.33333333]]
50000 data points are generated, which are:
Generated firings: [0 2 2 ... 0 3 3]
Generated states: [0 0 4 ... 5 5 5]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.09213609 0.11974287 0.28197556 0.06708212 0.17184695 0.2672164 ]
Estimated Gamma =  [[0.09875659 0.10620909 0.34048361 0.27622977 0.07868501 0.09963593]
 [0.1633682  0.4449762  0.10215722 0.1388858  0.04759492 0.1030

In [6]:
N = 3  # neurons
K = 7  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0])
gamma=np.array([[0.7,0.05,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.7,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.7,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.7,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.7,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.7,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.05,0.7]])
r=np.array([[1,0,1,0,1,1,1],
            [0,1,1,0,0,1,1],
            [1,0,0,1,1,0,1],
            [0,1,0,1,1,1,0]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=7)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

Data is generated according to an HMM with the following parameters:
Pi:
 [1 0 0 0 0 0 0]
Gamma:
 [[0.7  0.05 0.05 0.05 0.05 0.05 0.05]
 [0.05 0.7  0.05 0.05 0.05 0.05 0.05]
 [0.05 0.05 0.7  0.05 0.05 0.05 0.05]
 [0.05 0.05 0.05 0.7  0.05 0.05 0.05]
 [0.05 0.05 0.05 0.05 0.7  0.05 0.05]
 [0.05 0.05 0.05 0.05 0.05 0.7  0.05]
 [0.05 0.05 0.05 0.05 0.05 0.05 0.7 ]]
R:
 [[0.5        0.         0.5        0.         0.33333333 0.33333333
  0.33333333]
 [0.         0.5        0.5        0.         0.         0.33333333
  0.33333333]
 [0.5        0.         0.         0.5        0.33333333 0.
  0.33333333]
 [0.         0.5        0.         0.5        0.33333333 0.33333333
  0.        ]]
50000 data points are generated, which are:
Generated firings: [2 0 2 ... 1 3 3]
Generated states: [0 0 0 ... 5 4 4]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.20264149 0.14949601 0.0235667  0.2545672  0.04870783 0.18272025
 0.13830052]
Estimated Gamma =  [[0.34

In [7]:
N = 3  # neurons
K = 8  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0,0])
gamma=np.array([[0.72,0.04,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.72,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.72,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.72,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.72,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.72,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.72,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.72]])
r=np.array([[1,0,1,0,1,1,1,0],
            [0,1,1,0,0,1,1,1],
            [1,0,0,1,1,0,1,1],
            [0,1,0,1,1,1,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=8)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

Data is generated according to an HMM with the following parameters:
Pi:
 [1 0 0 0 0 0 0 0]
Gamma:
 [[0.72 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
 [0.04 0.72 0.04 0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.72 0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.72 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.72 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04 0.72 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04 0.04 0.72 0.04]
 [0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.72]]
R:
 [[0.5        0.         0.5        0.         0.33333333 0.33333333
  0.33333333 0.        ]
 [0.         0.5        0.5        0.         0.         0.33333333
  0.33333333 0.33333333]
 [0.5        0.         0.         0.5        0.33333333 0.
  0.33333333 0.33333333]
 [0.         0.5        0.         0.5        0.33333333 0.33333333
  0.         0.33333333]]
50000 data points are generated, which are:
Generated firings: [0 0 2 ... 0 2 1]
Generated states: [0 4 4 ... 6 6 2]
Rep 0...
##########################
Estimated values for the paramete